# Throughput
This notebook creates a throughput report for test results. It ties into the **Test Monitor Service** for retrieving filtered test results, the **Notebook Execution Service** for running outside of Jupyterhub, and the **Test Monitor Reports page** at #testmonitor/reports for displaying results.

The parameters and output use a schema recognized by the Test Monitor Reports page, which can be implemented by various report types. The Throughput notebook produces data that is best shown in a bar graph.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [1]:
import copy
import datetime
import pandas as pd
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service  
  Options: Any valid Test Monitor Results Dynamic Linq filter  
  Default: `'startedWithin <= "30.0:0:0"'`
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  Options: Day, System, Test Program, Operator, Part Number  
  Default: Day

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [2]:
results_filter = 'startedWithin <= "2.0:0:0"'
products_filter = ''

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [3]:
results_api = testmon.ResultsApi()

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter.

In [4]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT
)

results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)
results_list = [result.to_dict() for result in results]
#display(results_list)

### Create pandas dataframe
Put the data into a dataframe whose columns are test result id, start time, and group name.

In [5]:
df_results = pd.DataFrame.from_records(results_list)
df_results = df_results.join(df_results['status'].apply(pd.Series))
df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))
df_results.pop('status')
df_results.pop('status_type_summary')
properties = pd.json_normalize(df_results["properties"])
df_results = pd.concat([df_results.drop(["properties"], axis = 1), properties],axis =1 )
display(df_results)

,started_at,updated_at,program_name,id,system_id,host_name,operator,part_number,serial_number,total_time_in_seconds,...,UUT - BT MAC Address,Debug - PreUUTTimes1,Debug - PreUUTTimes2,Debug - PreUUTTimes3,Debug - PreUUTTimes4,Test Station SN Latest,nitmTestStandStartTime,Products Package Version,Test Station SN Range End,Test Station SN Range Start
0,2022-02-20 20:37:07.082217,2022-02-20 20:38:08.437891,CTS_Home_Analyzer_Manual.seq,c697894d-18eb-47cb-8e16-1120a8ca79b2,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-C6,CEQ0175,JEUHUS,M1000226,352354459,0.028321,...,-,"1 Test Pod idle: 22:33:31.184, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 6:37:02.973, TrueTrue5003False","4 Analyser loaded: 6:37:06.994, TrueTrue0True",352354459,2022-02-20T20:37:07.0650000Z,,369098751,352321536
1,2022-02-20 20:37:09.972302,2022-02-20 20:38:23.560395,CTS_Home_Analyzer_Manual.seq,ce9578e4-6be4-4cb1-b264-8acf0ea6fd73,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-C6,CEQ0175,JEUHUS,M1000226,352354460,17.187856,...,D4:A8:B4:E5:92:F9,"1 Test Pod idle: 22:33:36.669, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 6:37:06.190, TrueTrue5003False","4 Analyser loaded: 6:37:08.999, TrueTrue0True",352354460,2022-02-20T20:37:09.9620000Z,,369098751,352321536
2,2022-02-20 20:37:14.651153,2022-02-20 20:38:43.806890,CTS_Home_Analyzer_Manual.seq,19ccd9d2-b875-4a42-8059-51e30652bd6e,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-C6,CEQ0175,JEUHUS,M1000226,352354461,17.911268,...,C9:C5:76:7F:D4:BE,"1 Test Pod idle: 22:33:40.423, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 6:37:09.100, TrueTrue5003False","4 Analyser loaded: 6:37:14.311, TrueTrue0True",352354461,2022-02-20T20:37:14.6420000Z,,369098751,352321536
3,2022-02-20 20:37:16.510815,2022-02-20 20:38:44.011391,CTS_Home_Analyzer_Manual.seq,451b957a-9f49-44fa-aff3-b679a1548133,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-C6,CEQ0175,JEUHUS,M1000226,352354462,16.913830,...,DA:6C:40:66:8B:D3,"1 Test Pod idle: 22:33:43.938, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 6:37:14.311, TrueTrue5003False","4 Analyser loaded: 6:37:15.514, TrueTrue0True",352354462,2022-02-20T20:37:16.5010000Z,,369098751,352321536
4,2022-02-20 20:37:22.849411,2022-02-20 20:38:56.077390,CTS_Home_Analyzer_Manual.seq,d27a734f-ac50-45a7-8758-0971fc1d5560,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-C6,CEQ0175,JEUHUS,M1000226,352354463,17.315277,...,C6:FE:26:2C:31:A5,"1 Test Pod idle: 6:37:14.302, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 6:37:19.64, TrueTrue0False","4 Analyser loaded: 6:37:21.871, TrueTrue0True",352354463,2022-02-20T20:37:22.8400000Z,,369098751,352321536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230,2022-02-21 03:51:00.285126,2022-02-21 03:51:49.318028,CTS_Home_Analyzer_Manual.seq,acfd78ce-eec8-4599-9c50-d2d95add81b1,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-F8,CEQ0181,APATEL,M1000226,469793800,0.000000,...,EA:57:8E:4C:A0:5E,"1 Test Pod idle: 13:50:54.855, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 13:50:56.508, TrueTrue0False","4 Analyser loaded: 13:50:59.317, TrueTrue0True",469793800,2022-02-21T03:51:00.2780000Z,,486539263,469762048
20231,2022-02-21 03:51:08.700290,2022-02-21 03:51:48.195027,CTS_Home_Analyzer_Manual.seq,5030bb51-4326-4d13-af19-5480153c6199,P-2100--SN-Default_string--MAC-2C-94-64-03-C8-D0,CEQ0180,APATEL,M1000226,486570979,17.379552,...,CE:2D:2C:08:87:50,"1 Test Pod idle: 13:50:57.603, TrueFalse0False","2 Start testing: Not executed, 2.1 Clear Histo...","3 Device Busy: 13:51:04.284, TrueTrue0False","4 Analyser loaded: 13:51:07.389, TrueTrue0True",486570979,2022-02-21T03:51:08.6930000Z,,503316479,486539264
20232,2022-02-21 03:51:12.067413,2022-02-21 03:51:52.232528,CTS_Home_Analyzer_Manual.seq,67b71d3e-a096-4299-b262-5339eb1f

### KPI calculations
SN usage percentage : (latest_SN - Start_SN) / (End_SN - Start_SN) *100

In [6]:
#conver to int
df_SN_Start = pd.to_numeric(df_results['TS SN Start'],errors='ignore')
df_SN_End = pd.to_numeric(df_results['TS SN End'],errors='ignore')
df_SN_Latest = pd.to_numeric(df_results['TS SN Current'],errors='ignore')

#calcultion
df_SN_percentage = (df_SN_Latest-df_SN_Start) / (df_SN_End - df_SN_Start) *100
df_SN_percentage.name='SN_Percentage'

#merge into table
df_results = pd.concat([df_results, df_SN_percentage], axis = 1)

#display(df_results)

KeyError: 'TS SN Start'

### Convert the dataframe to the SystemLink reports output format
The result format for a SystemLink report consists of a list of output objects as defined below:
- `type`: The type of the output. Accepted values are 'data_frame' and 'scalar'.
- `id`: Corresponds to the id specified in the 'output' metadata. Used for returning multiple outputs with the 'V2' report format.
- `data`: A dict representing the 'data_frame' type output data.
    - `columns`: A list of dicts containing the names and data type for each column in the dataframe.
    - `values`: A list of lists containing the dataframe values. The sublists are ordered according to the 'columns' configuration.
- `value`: The value returned for the 'scalar' output type.
- `config`: The configurations for the given output.
    - `title`: The output title.
    - `graph`: The graph configurations.
        - `axis_labels`: The x-axis label and y-axis label.
        - `plots`: A list of plots to display mapped from the dataframe's columns, along with configuration options.
            - `x`: The dataframe column corresponding to the x-axis values.
            - `y`: The dataframe column corresponding to the y-axis values.
            - `style`: The plot's style. Accepted values are ['LINE', 'BAR', 'SCATTER'].
            - `color`: The plot's color. Accepted formats are ['blue', '#0000ff', 'rbg(0,0,255)'].
            - `label`: The plot's name, to be shown in a plot legend. 
            - `secondary_y`: Whether or not to display this plot on a second y-axis.
            - `group_by`: A list of columns in the dataframe on which to group data, e.g. to color individual points.
        - `orientation`: 'HORIZONTAL' or 'VERTICAL'.
        - `stacked`: Whether or not to display the plots stacked on top of each other.

Here is an example of a notebook result with two outputs, one of which is a dataframe with two columns, and the other is a scalar value:
```
[{
    'type': 'data_frame',
    'id': 'output_id_1',
    'data': {
        'columns': [
            {'name': 'time', 'type': 'datetime'},
            {'name': 'value', 'type': 'number'}
         ],
        'values': [
            ['2020-09-29T00:00:00.000Z', 46.1538461538],
            ['2020-09-30T00:00:00.000Z', 63.1578947368],
            ...
         ]
    },
    'config': {
        'title': 'My Title',
        'graph': {
            'axis_labels': ['X Axis', 'Y Axis'],
            'orientation': 'VERTICAL',
            'plots': [
                {'x': 'time', 'y': 'value', 'style': 'BAR', 'color': '#0000ff', 'label': 'Plot 1'}
            ]
        }
    }
}, {
    'type': 'scalar',
    'id': 'output_id_2',
    'config': {
        'title': 'My Title'
    },
    'value': 5
}]
```

For this report, there is one output, which is a dataframe with two columns. For a grouping of 'Day', the first column contains ISO-8601 date strings. For any other grouping option, the first column contains categorical string values. The second column contains numerical values representing the throughput.

| started_at                 | throughput    |
|----------------------------|---------------|
| '2020-09-29'               | 23            |
| '2020-09-30'               | 45            |
| '2020-10-01'               | 30            |

The graph configuration specifies a single plot, where the x-axis is the group values and the y-axis is the throughput. We use Pandas to convert the dataframe built in the previous cells into a tabular format and then return that with the result object.

In [ ]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_results, index=False)['fields'],
    'values': df_results.values.tolist(),
}

results_graph = {
    'type': 'data_frame',
    'id': 'sakal',
    'data': df_dict
}

result = [results_graph]

### Record results with Scrapbook

In [ ]:
sb.glue('result', result)

In [ ]:
result